In [1]:
import requests
import pandas as pd
import time

# Webflow API token and collection ID
WEBFLOW_API_TOKEN = "fc1ebf4ad07126c49afd63877c7de3c2143a226e2f708b3dbf127ae0b6aaa0d8"
COLLECTION_ID = "6660d3a96fe3b376c162563e"
API_URL = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}/items"

headers = {
    "Authorization": f"Bearer {WEBFLOW_API_TOKEN}",
    "accept": "application/json"
}

all_items = []
offset = 0
limit = 100

while True:
    response = requests.get(f"{API_URL}?offset={offset}&limit={limit}", headers=headers)
    
    if response.status_code != 200:
        print(f"❌ Error fetching data: {response.status_code} - {response.text}")
        break

    data = response.json()
    items = data.get("items", [])

    all_items.extend(items)
    print(f"✅ Retrieved {len(items)} items (offset {offset})")

    if len(items) < limit:
        break  # No more pages

    offset += limit
    time.sleep(0.2)  # polite delay to avoid rate limiting

# Convert to DataFrame
df = pd.json_normalize(all_items)

✅ Retrieved 100 items (offset 0)
✅ Retrieved 100 items (offset 100)
✅ Retrieved 100 items (offset 200)
✅ Retrieved 100 items (offset 300)
✅ Retrieved 36 items (offset 400)


In [67]:
import requests

WEBFLOW_API_TOKEN = "fc1ebf4ad07126c49afd63877c7de3c2143a226e2f708b3dbf127ae0b6aaa0d8"
COLLECTION_ID = "6660d3a96fe3b376c162563e"

url = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}"
headers = {
    "Authorization": f"Bearer {WEBFLOW_API_TOKEN}",
    "accept-version": "1.0.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    cms_locale_id = data.get("rootCmsLocaleId")
    print(f"✅ rootCmsLocaleId: {cms_locale_id}")
else:
    print(f"❌ Error {response.status_code}: {response.text}")


✅ rootCmsLocaleId: None


In [44]:
df.columns

Index(['id', 'cmsLocaleId', 'lastPublished', 'lastUpdated', 'createdOn',
       'isArchived', 'isDraft', 'fieldData.descuento',
       'fieldData.descuento-2x1', 'fieldData.descuento-3x2',
       'fieldData.productoreserva', 'fieldData.descuento-off',
       'fieldData.name', 'fieldData.slug', 'fieldData.precio',
       'fieldData.imagen-del-producto.fileId',
       'fieldData.imagen-del-producto.url',
       'fieldData.imagen-del-producto.alt', 'fieldData.categoria',
       'fieldData.descripcion', 'fieldData.pasillo', 'fieldData.categories',
       'fieldData.precios', 'fieldData.precio-descuento', 'fieldData.ocasion',
       'fieldData.temperatura-de-servicio', 'fieldData.tipo',
       'fieldData.denominacion', 'fieldData.ciudad', 'fieldData.peso-volumen',
       'fieldData.gr-ml', 'fieldData.maridaje-1', 'fieldData.maridaje-2',
       'fieldData.bodega', 'fieldData.notas-de-cata',
       'fieldData.cepa-principal', 'fieldData.region'],
      dtype='object')

In [68]:
import pandas as pd
import requests
import time

# Load CSV
df = pd.read_csv(r"C:\Users\dhernandez\Downloads\Maestri Milano - Products.csv")
df = df.replace([float("inf"), float("-inf"), pd.NA], '')

ACCESS_TOKEN = "fc1ebf4ad07126c49afd63877c7de3c2143a226e2f708b3dbf127ae0b6aaa0d8"
COLLECTION_ID = "6660d3a96fe3b376c162563e"
API_URL = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}/items"
PUBLISH_URL_TEMPLATE = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}/items/{{item_id}}/publish"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "accept": "application/json",
    "content-type": "application/json"
}

category_mapping = {
    "cristaleria": "68426551cb9513e04fb7b846",
    "cajas": "66834c5bb2f187cbd0bc92de",
    "maestri-wine-club": "6663699bebde7a336c1c0fe5",
    "vinos-de-la-casa": "66635db36d9753905f695788",
    "anchetas": "6660caffc04494865e3bce5f",
    "panettone": "6660caf7ff352dfda277bca5",
    "pasta": "6660cae85768b4de8082734e",
    "galletas-italianas": "6660cad2b9bdd7371192e27a",
    "carnes": "6660cab03742b98ef120b0ac",
    "quesos": "6660ca3823741b8f165309d3",
    "vinos": "6660ca3823741b8f165309d1",
    "alimentos-gourmet": "6660ca3823741b8f165309cb",
    "trufas": "6660ca3823741b8f165309c9",
    "charcuteria": "6660ca3823741b8f165309c4"
}

for idx, row in df.iterrows():
    try:
        category_slug = str(row.get("Categories", "")).strip().lower()
        payload = {
            "isArchived": row.get("Archived", False),
            "isDraft": bool(row.get("Draft", False)),
            "fieldData": {
                "name": str(row["Nombre"]).strip(),
                "slug": str(row["Slug"]).strip(),
                "precio": str(int(row["Precio"])) if not pd.isna(row["Precio"]) else "0",
                "imagen-del-producto": str(row.get("Imagen del Producto", "")).strip(),
                "categoria": str(row.get("Categoria", "")).strip(),
                "descripcion": str(row.get("Descripción", "")).strip(),
                "maridaje-1": str(row.get("Maridaje 1", "")).strip(),
                "maridaje-2": str(row.get("Maridaje 2", "")).strip(),
                "notas-de-cata": str(row.get("Notas de cata", "")).strip(),
                "temperatura-de-servicio": str(row.get("Temperatura de servicio", "")).strip(),
                "pasillo": str(row.get("Pasillo", "")).strip(),
                "tipo": str(row.get("Tipo", "")).strip(),
                "region": str(row.get("Región", "")).strip(),
                "cepa-principal": str(row.get("Cepa Principal", "")).strip(),
                "ocasion": str(row.get("Ocasión", "")).strip(),
                "bodega": str(row.get("Bodega", "")).strip(),
                "denominacion": str(row.get("Denominación", "")).strip(),
                "peso-volumen": str(row.get("Peso / Volumen", "")).strip(),
                "gr-ml": str(row.get("Gr/ml", "")).strip(),
                "ciudad": str(row.get("Ciudad", "")).strip(),
                "productoreserva": bool(row.get("ProductoReserva", False)),
                "descuento": bool(row.get("Descuento", False)),
                "descuento-2x1": bool(row.get("Descuento 2x1", False)),
                "descuento-3x2": bool(row.get("Descuento 3x2", False)),
                "descuento-off": bool(row.get("Descuento%off", False)),
                "categories": category_mapping.get(category_slug)
            }
        }

        res = requests.post(API_URL, headers=headers, json=payload)

        if res.status_code in [200, 201]:
            data = res.json()
            item_id = data.get("id")
            print(f"✅ Created: {row['Nombre']} — ID: {item_id}")

            # Publish the item (optional but needed for `live=true` visibility)
            publish_url = PUBLISH_URL_TEMPLATE.format(item_id=item_id)
            pub = requests.post(publish_url, headers=headers)
            if pub.status_code == 200:
                print(f"🚀 Published: {row['Nombre']}")
            else:
                print(f"⚠️ Failed to publish {row['Nombre']}: {pub.status_code} {pub.text}")
        else:
            print(f"❌ Error on row {idx} ({res.status_code}): {res.text}")

        time.sleep(0.3)

    except KeyError as ke:
        print(f"⚠️ Missing key on row {idx}: {ke}")
    except Exception as e:
        print(f"❌ Exception on row {idx}: {e}")


❌ Error on row 0 (202): {"id":"68844f0331a5184b6ce8166e","cmsLocaleId":"664d95581633c04b8a3808b5","lastPublished":null,"lastUpdated":"2025-07-26T03:44:03.628Z","createdOn":"2025-07-26T03:44:03.628Z","isArchived":false,"isDraft":false,"fieldData":{"productoreserva":false,"descuento":false,"descuento-2x1":false,"descuento-3x2":false,"descuento-off":false,"maridaje-1":null,"maridaje-2":null,"notas-de-cata":null,"temperatura-de-servicio":null,"pasillo":null,"tipo":null,"region":null,"cepa-principal":null,"ocasion":null,"bodega":null,"denominacion":null,"name":"Pecorino Cafone","precio":"49000","categoria":"Quesos","descripcion":"<p id=\"\">Queso pecorino de leche de oveja con 45 días de maduración. Ideal para preparación donde se quiere derretir el producto. Peso 350 Gr</p><p id=\"\">‍</p>","peso-volumen":"350","gr-ml":"Gramo a $140","ciudad":"Bogotá","imagen-del-producto":{"fileId":"688447039b8aeb3a1e3160fb","url":"https://cdn.prod.website-files.com/664d95581633c04b8a3808d3/688447039b8aeb

In [38]:
import pandas as pd
import requests

WEBFLOW_API_TOKEN = "026a04fef179155b6a04fbfd49e07c722e7621b91ad98961f6f298987c070180"
COLLECTION_ID = "6660d3a96fe3b376c162563e"
BASE_URL = f"https://api.webflow.com/v2/collections/{COLLECTION_ID}/items"

API_URL = "https://api.webflow.com/v2"

# Load the data
df2 = pd.read_csv(r"C:\Users\dhernandez\Downloads\Maestri Milano - Products.csv")
df2 =df2[(df2["Archived"] == True) | (df2["Draft"] == True)]


# Merge by name
merged = pd.merge(df2, df, left_on="Slug", right_on="fieldData.slug", how="inner")
merged = merged[["id", "fieldData.slug", "Archived", "Draft"]]

# # Filter rows where Archived or Draft is True
# to_unpublish = merged[(merged["Archived"] == True) | (merged["Draft"] == True)]
to_unpublish = merged.drop_duplicates(subset="id")


headers = {
    "Authorization": f"Bearer {WEBFLOW_API_TOKEN}",
    "accept": "application/json"
}

for _, row in to_unpublish.iterrows():
    collection_id = '6660d3a96fe3b376c162563e'
    item_id = row["id"]
    url = f"{API_URL}/collections/{collection_id}/items/{item_id}/live"

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"✅ Unpublished: {item_id}")
    else:
        print(f"⚠️ Failed to unpublish {item_id} — {response.status_code}: {response.text}")


✅ Unpublished: 6884291d58e81e3c2b6453a0
⚠️ Failed to unpublish 6884291d6af475a86bde8631 — 404: {"message":"Requested resource not found","code":"resource_not_found","externalReference":null,"details":[]}
✅ Unpublished: 6884291e40ff0915c9f321be
✅ Unpublished: 6884291f938c163faf2efef3
✅ Unpublished: 6884291e8d241af6e8c4d3e2
✅ Unpublished: 6884291e1edcf2430cec94fb
✅ Unpublished: 6884291fea775e47674bb81c
✅ Unpublished: 6884291fd4249589705b949c
✅ Unpublished: 6884291ff74b709059fe7a27
✅ Unpublished: 6884291f50d006b994253212
✅ Unpublished: 6884292059c4754926a7e0f6
✅ Unpublished: 68842920f3e9d34be5aba398
✅ Unpublished: 68842920f37d447a2d91747b
✅ Unpublished: 68842920d2ddf3c78d6794f0
✅ Unpublished: 68842920b1238eebfb376fd3
✅ Unpublished: 68842921fd2175c6335547b5
✅ Unpublished: 68842921f5f493b2bb58e9f8
✅ Unpublished: 688429217b5146dd43b766b1
✅ Unpublished: 68842922f78c65307a658b10
✅ Unpublished: 68842922b18ab1756c53776f
✅ Unpublished: 688429234a8c450cc90f56ed
✅ Unpublished: 688429239e17940145358

In [29]:
to_unpublish

,Nombre,Slug,Collection ID,Locale ID,Item ID,Archived,Draft,Created On,Updated On,Published On,...,fieldData.descripcion,fieldData.notas-de-cata,fieldData.categoria,fieldData.cepa-principal,fieldData.region,fieldData.categories,fieldData.slug,fieldData.imagen-del-producto.fileId,fieldData.imagen-del-producto.url,fieldData.imagen-del-producto.alt
1,"""Timperosse"""" Petit Verdot - Terre Siciliane Igt",timperosse-petit-verdot---terre-siciliane-igt,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,66615040ba2c5de34c972281,True,False,Thu Jun 06 2024 05:59:28 GMT+0000 (Coordinated...,Mon Jun 17 2024 02:12:37 GMT+0000 (Coordinated...,NaN,...,"<p id="""">ELABORACIÓN DEL VINO Los viñedos ubic...",Color rojo rubí intenso con reflejos violáceos...,Vinos y licores,Petit Verdot,Sicilia,6660ca3823741b8f165309d1,timperosse-petit-verdot---terre-siciliane-igt,6884291d58e81e3c2b64539b,https://cdn.prod.website-files.com/664d9558163...,NaN
5,ARMA TU ANCHETA,arma-tu-ancheta,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,66615044b325a9bc9fc28303,False,True,Thu Jun 06 2024 05:59:32 GMT+0000 (Coordinated...,Fri Jun 07 2024 21:04:03 GMT+0000 (Coordinated...,NaN,...,,None,Alimentos > Sin categorizar,None,None,NaN,arma-tu-ancheta,NaN,NaN,NaN
6,Aceite EVO Aromatizado a la trufa blanca 100ml,aceite-evo-aromatizado-a-la-trufa-blanca-100ml,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,6661504abf4320c05e9e3668,True,True,Thu Jun 06 2024 05:59:38 GMT+0000 (Coordinated...,Wed Jul 02 2025 21:01:51 GMT+0000 (Coordinated...,NaN,...,"<p id="""">Un aceite obtenido con las aceitunas ...",None,Trufas,None,None,6660ca3823741b8f165309c9,aceite-evo-aromatizado-a-la-trufa-blanca-100ml,6884291e40ff0915c9f321a9,https://cdn.prod.website-files.com/664d9558163...,NaN
7,Aceite EVO Aromatizado a la trufa blanca 250ml,aceite-evo-aromatizado-a-la-trufa-blanca-250ml,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,6661504a981f49b444eee695,True,False,Thu Jun 06 2024 05:59:38 GMT+0000 (Coordinated...,Sat Feb 08 2025 21:23:06 GMT+0000 (Coordinated...,NaN,...,"<p id="""">Un aceite obtenido con las aceitunas ...",None,Trufas,None,None,6660ca3823741b8f165309c9,aceite-evo-aromatizado-a-la-trufa-blanca-250ml,6884291e40ff0915c9f321a9,https://cdn.prod.website-files.com/664d9558163...,NaN
8,Aceite EVO Aromatizado a la trufa negra 250ml,aceite-evo-aromatizado-a-la-trufa-negra-250ml,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,6661504bbc6a057dc9d92f75,True,True,Thu Jun 06 2024 05:59:39 GMT+0000 (Coordinated...,Sat Feb 08 2025 21:25:08 GMT+0000 (Coordinated...,NaN,...,"<p id="""">Un aceite obtenido con las aceitunas ...",None,Trufas,None,None,6660ca3823741b8f165309c9,aceite-evo-aromatizado-a-la-trufa-negra-250ml,6884291e8d241af6e8c4d3dd,https://cdn.prod.website-files.com/664d9558163...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,Vinagre Balsámico,vinagre-balsamico-di-modena-igp,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,66615042a0a5c371bca64065,True,False,Thu Jun 06 2024 05:59:30 GMT+0000 (Coordinated...,Wed Jan 29 2025 22:10:17 GMT+0000 (Coordinated...,NaN,...,"<p id="""">El equilibrio adecuado de &nbsp;sensa...",None,Alimentos Gourmet,None,None,6660ca3823741b8f165309cb,vinagre-balsamico-di-modena-igp,688429525f035929851d8d29,https://cdn.prod.website-files.com/664d9558163...,NaN
364,Vinagre Balsámico Di Modena IGP Aroma e Frescheza,vinagre-balsamico-di-modena-igp-aroma-e-frescheza,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,666150439507389a4b5866d8,True,False,Thu Jun 06 2024 05:59:31 GMT+0000 (Coordinated...,Fri Dec 13 2024 20:12:06 GMT+0000 (Coordinated...,NaN,...,,None,Alimentos Gourmet,None,None,6660ca3823741b8f165309cb,vinagre-balsamico-di-modena-igp-aroma-e-frescheza,6884295250d006b99425368f,https://cdn.prod.website-files.com/664d9558163...,NaN
365,Vinagre Super añejo,vinagre-balsamico-di-modena-essenza-igp---inve...,6660d3a96fe3b376c162563e,664d95581633c04b8a3808b5,666150429507389a4b58656e,True,True,Thu Jun 06 